#**Import and Mount and GPU**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np                                              
import pandas as pd                                             
import matplotlib.pyplot as plt                                 
import torchvision                
import torchvision.transforms as transforms              
from torch.utils.data import SubsetRandomSampler 
import torch.utils.data as data   
import torch
import torch.nn as nn
from torchsummary import summary
import torch.nn.functional as F
import sys
from time import time
sys.path.append('/content/gdrive/MyDrive/ECE324 project/Additional files/Data_Loader.py')
sys.path.append('/content/gdrive/My Drive/ECE324 project/Additional files')
sys.path.append('/content/gdrive/My Drive/ECE324 project/Clean Data')
sys.path.append('/content/gdrive/MyDrive/ECE324 project/test_varied_bg')
sys.path.append('/content/gdrive/MyDrive/ECE324 project/test_white_bg')
from Data_Loader import *
from sklearn.metrics import confusion_matrix, classification_report
from torchvision import models


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

#**Loading in the Data**


In [ ]:

my_transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Resize((224,224)),
  transforms.Normalize(mean = [0.66445047,0.55465436,0.447036], std = [0.321551,0.33547384,0.3524585])
  ])

dataset =  torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/ECE324 project/Clean Data', transform=my_transform)
print(len(dataset))
train_data, valid_data, test_data = torch.utils.data.random_split(dataset, [9600,2400,3000], generator=torch.Generator().manual_seed(0))

train_loader = DataLoader(train_data, batch_size=64,shuffle= True)                  
valid_loader = DataLoader(valid_data, batch_size=64,shuffle= True)
test_loader = DataLoader(test_data, batch_size=64,shuffle= True)

def relabel_state(argument):
    switcher = {
        0: 0,
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 1,
        6: 1,
        7: 1,
        8: 1,
        9: 1,
        10:2,
        11:2,
        12:2,
        13:2,
        14:2}
    return switcher[argument.item()]

def relabel_type(argument):
    switcher = {
        0: 0,
        1: 1,
        2: 2,
        3: 3,
        4: 4,
        5: 0,
        6: 1,
        7: 2,
        8: 3,
        9: 4,
        10:0,
        11:1,
        12:2,
        13:3,
        14:4}
    return switcher[argument.item()]

def recombine_prediction(argument1,argument2):
    switcher = {
         str([0,0]): 0,
         str([0,1]): 1,
         str([0,2]): 2,
         str([0,3]): 3,
         str([0,4]): 4,
         str([1,0]): 5,
         str([1,1]): 6,
         str([1,2]): 7,
         str([1,3]): 8,
         str([1,4]): 9,
         str([2,0]):10,
         str([2,1]):11,
         str([2,2]):12,
         str([2,3]):13,
         str([2,4]):14}
    return switcher[str([argument1.item(),argument2.item()])]


# **Various Multi head - CNN architectures**



In [ ]:
# CNN Model

# for 15,5--> 30,5 use 85720
# for 30,5,30,5 --> 84270
#30,10    30,10 --> 72030
#30,10    50,10 -->120050
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    # Convolution layers
    self.conv1 = nn.Conv2d(3,30,10) # 6 kernels, 5x5
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(30,50,20) # 16 kernels, 5x5

    # MLP State
    self.fc1 = nn.Linear(96800,500)
    self.fc2 = nn.Linear(500,120)
    self.fc3 = nn.Linear(120,3)

    # MLP Type
    self.fc4 = nn.Linear(96800,300)#44944
    self.fc5 = nn.Linear(300,100)
    self.fc6 = nn.Linear(100,5)

    #self.drop = nn.Dropout(p=0.5)
    #self.drop_conv = nn.Dropout(p=0.1)
    self.Bn1 = nn.BatchNorm2d(30)
    self.Bn2 = nn.BatchNorm2d(50)

  def forward(self,x):
    x = self.pool(F.relu(self.Bn1(self.conv1(x))))
    x = self.pool(F.relu(self.Bn2(self.conv2(x))))
    #x = self.pool(F.relu(self.conv1(x)))
    #x = self.pool(F.relu(self.conv2(x)))
    x = x.view(x.size(0), -1)
    #print(x.size())

    # State head
    x1 = F.relu(self.fc1(x))
    x1 = F.relu(self.fc2(x1))
    x1 = self.fc3(x1)

    # Type head
    x2 = F.relu(self.fc4(x))
    x2 = F.relu(self.fc5(x2))
    x2 = self.fc6(x2)

    return x1, x2

# **Early stopping code from github**

In [ ]:
import numpy as np
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

#**Instantiate arguments and parameters**


In [ ]:
#MODEL ARGUMENTS
lr = 0.03
epochs = 50
eval_every = 10
model = CNN()

model.to(device)


optimizer = torch.optim.SGD(model.parameters(), lr=lr)
loss_fnc= torch.nn.CrossEntropyLoss()

loss_fnc.to(device)


#OUTPUT DATA ARGUMENTS
seed = 1
torch.manual_seed(seed)
save= True
confusion=True
plot = True
statistics = True

### **Training loop using train and validation**

In [ ]:
#Prepare plotting lists
train_loss_list = []
valid_loss_list = []
test_loss_list = []
train_acc_list = []
valid_acc_list = []
test_acc_list = []

train_loss_list_state = []
valid_loss_list_state = []
test_loss_list_state = []
train_acc_list_state = []
valid_acc_list_state = []
test_acc_list_state = []

train_loss_list_type = []
valid_loss_list_type = []
test_loss_list_type = []
train_acc_list_type = []
valid_acc_list_type = []
test_acc_list_type = []


epoch_list = []         #Epoch number
start = time()          #Time training process

for e in range(epochs):
    running_loss = []
    running_accuracy = []
    running_loss_state = []
    running_accuracy_state = []
    running_loss_type = []
    running_accuracy_type = []


    running_valid_loss = []
    running_valid_accuracy = []
    running_valid_loss_state = []
    running_valid_accuracy_state = []
    running_valid_loss_type = []
    running_valid_accuracy_type = []
    print("going thru data , epoch {}".format(e))
  #               LOOKING AT TRAINING DATA

    for i, data in enumerate(train_loader):

        inputs, label = data[0].to(device),data[1].to(device)

        label_state = torch.zeros(len(label)).to(device)
        label_type = torch.zeros(len(label)).to(device)

        for k in range(len(label)):
          label_state[k] = relabel_state(label[k])
          label_type[k] = relabel_type(label[k])

            #setting gradients to zero and running model on batch
        optimizer.zero_grad()
        predictions_state, predictions_type = model(inputs)

            #computing loss based on Cross Entropy
        batch_loss_state = loss_fnc(input=predictions_state.squeeze(), target=label_state.long())
        batch_loss_type = loss_fnc(input=predictions_type.squeeze(), target=label_type.long())

        #Combining losses and calculating gradients
        Overall_loss = batch_loss_state + batch_loss_type
        Overall_loss.backward()
        optimizer.step()
        #evaluating Training Acc
        _, predicted_state = torch.max(predictions_state.data, 1)
        _,predicted_type = torch.max(predictions_type.data, 1)
        trainAcc_state = (label_state == predicted_state).sum().item() / 64
        trainAcc_type = (label_type == predicted_type).sum().item() /64

        if i % eval_every == 0:
            print("epoch: {} |||   loss type:   {}  trainAcc type:   {} |||  loss state {}  trainAcc state:  {}  ".format(e, batch_loss_type, trainAcc_type, batch_loss_state, trainAcc_state))

        running_loss.append(Overall_loss.item())
        running_accuracy.append((trainAcc_state+trainAcc_type)/2)
        running_loss_state.append(batch_loss_state.item())
        running_accuracy_state.append(trainAcc_state)
        running_loss_type.append(batch_loss_type.item())
        running_accuracy_type.append(trainAcc_type)
        del Overall_loss
        del batch_loss_state
        del batch_loss_type
        del label_state
        del label_type
        del predictions_state
        del predictions_type

        
    #               LOOKING AT Validation DATA
    for j, data_v in enumerate(valid_loader):
            #get batch of data
        inputs_v, label_v = data_v[0].to(device),data_v[1].to(device)
    
        label_state_v = torch.zeros(len(label_v)).to(device)
        label_type_v = torch.zeros(len(label_v)).to(device)

        #inputs_v, label_v = data_v
    
        #label_state_v = torch.zeros(len(label_v))
        #label_type_v = torch.zeros(len(label_v))
        for a in range(len(label_v)):
          label_state_v[a] = relabel_state(label_v[a])
          label_type_v[a] = relabel_type(label_v[a])

          #run model on validation batch
        predictions_state_v, predictions_type_v = model(inputs_v)

            #compute loss
        batch_valid_loss_state = loss_fnc(input=predictions_state_v.squeeze(), target=label_state_v.long())
        batch_valid_loss_type = loss_fnc(input=predictions_type_v.squeeze(), target=label_type_v.long())

        Overall_loss_v = batch_valid_loss_state + batch_valid_loss_type

            #evaluate
        _, predicted_state_v = torch.max(predictions_state_v.data, 1)
        _,predicted_type_v = torch.max(predictions_type_v.data, 1)

        validAcc_state = (label_state_v == predicted_state_v).sum().item() / 64
        validAcc_type = (label_type_v == predicted_type_v).sum().item() / 64

        if j % eval_every == 0:
             print("epoch: {} |||   vloss type:   {}  validAcc type:   {} |||  vloss state {}  validAcc state:  {}  ".format(e, batch_valid_loss_type, validAcc_type, batch_valid_loss_state,validAcc_state))
        
        running_valid_loss.append(Overall_loss_v.item())
        running_valid_accuracy.append((validAcc_state+validAcc_type)/2)
    
        running_valid_loss_state.append(batch_valid_loss_state.item())
        running_valid_accuracy_state.append(validAcc_state)
        running_valid_loss_type.append(batch_valid_loss_type.item())
        running_valid_accuracy_type.append(validAcc_type)

        del Overall_loss_v
        del batch_valid_loss_state
        del batch_valid_loss_type
        del label_state_v
        del label_type_v
        del predictions_state_v
        del predictions_type_v
        #--------------Storing data from every epoch into lists--------------

    #             lists for TRAINING accuracy
    #Overall accuracy
    trainacc_ = sum(running_accuracy) / float(len(running_accuracy))
    train_acc_list.append(trainacc_)
    #state accuracy
    trainacc_state_ = sum(running_accuracy_state) / float(len(running_accuracy_state))
    train_acc_list_state.append(trainacc_state_)
    #type accuracy
    trainAcc_type_ = sum(running_accuracy_type) / float(len(running_accuracy_type))
    train_acc_list_type.append(trainAcc_type_)

    #             lists for TRAINING losses
    #Overall Loss
    loss_ = sum(running_loss) / float(len(running_loss))
    train_loss_list.append(loss_)
    #State Loss
    loss_state_ = sum(running_loss_state) / float(len(running_loss_state))
    train_loss_list_state.append(loss_state_)
    #Type Loss
    loss_type_ = sum(running_loss_type) / float(len(running_loss_type))
    train_loss_list_type.append(loss_type_)


    #------------------------------------------------------------------------------------
    #             lists for VALIDATION accuracy
    #Overall accuracy
    validacc_ = sum(running_valid_accuracy) / float(len(running_valid_accuracy))
    valid_acc_list.append(validacc_)
    #state accuracy
    validacc_state_ = sum(running_valid_accuracy_state) / float(len(running_valid_accuracy_state))
    valid_acc_list_state.append(validacc_state_)
    #type accuracy
    validAcc_type_ = sum(running_valid_accuracy_type) / float(len(running_valid_accuracy_type))
    valid_acc_list_type.append(validAcc_type_)

    #             lists for VALIDATION losses
    #Overall Loss
    valid_loss_ = sum(running_valid_loss) / float(len(running_valid_loss))
    valid_loss_list.append(valid_loss_)
    #State Loss
    valid_loss_state_ = sum(running_valid_loss_state) / float(len(running_valid_loss_state))
    valid_loss_list_state.append(valid_loss_state_)
    #Type Loss
    valid_loss_type_ = sum(running_valid_loss_type) / float(len(running_valid_loss_type))
    valid_loss_list_type.append(valid_loss_type_)

        # early_stopping needs the validation loss to check if it has decresed, 
        # and if it has, it will make a checkpoint of the current model
    a=EarlyStopping(valid_loss_, model)
        
    if a.early_stop == True:
        print("Early stopping")
        break

    epoch_list.append(e)

end = time()

#Quick training stats
print("Total training time: ", end - start)
print("Epochs trained for:",len(epoch_list))
print("Max train Acc: ", max(train_acc_list))
print("Max valid Acc: ", max(valid_acc_list))


### **PLOTTING**

In [ ]:
if plot == True:
        #Plot overall loss vs epoch
        plt.plot(epoch_list, train_loss_list, label='Train')
        plt.plot(epoch_list, valid_loss_list, label='Valid')
        plt.title('Overall Loss vs. Epoch')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.show()

        #Plot overall Accuracy vs epoch
        plt.plot(epoch_list, train_acc_list, label='Train')
        plt.plot(epoch_list, valid_acc_list, label='Valid')
        plt.title('Training Accuracy vs. Epoch')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()

                #Plot TYPE loss vs epoch
        plt.plot(epoch_list, train_loss_list_type, label='Train')
        plt.plot(epoch_list, valid_loss_list_type, label='Valid')
        plt.title('Type Loss vs. Epoch')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.show()

                #Plot TYPE Accuracy vs epoch
        plt.plot(epoch_list, train_acc_list_type, label='Train')
        plt.plot(epoch_list, valid_acc_list_type, label='Valid')
        plt.title('Type Accuracy vs. Epoch')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()

                #Plot STATE loss vs epoch
        plt.plot(epoch_list, train_loss_list_state, label='Train')
        plt.plot(epoch_list, valid_loss_list_state, label='Valid')
        plt.title('State Loss vs. Epoch')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.show()

                #Plot STATE Accuracy vs epoch
        plt.plot(epoch_list, train_acc_list_state, label='Train')
        plt.plot(epoch_list, valid_acc_list_state, label='Valid')
        plt.title('State Accuracy vs. Epoch')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()



### **Saving model**

In [ ]:
if save == True:
    torch.save(model,'/content/drive/MyDrive/ECE324 project/Figures/Multi-Head/row 20/model.pt')

### **Confusion Matrix**


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
def confusion_classification_report(model,dataloader):
    predlist_state=torch.zeros(0,dtype=torch.long, device='cpu')
    lbllist_state=torch.zeros(0,dtype=torch.long, device='cpu')
    predlist_type=torch.zeros(0,dtype=torch.long, device='cpu')
    lbllist_type=torch.zeros(0,dtype=torch.long, device='cpu')
    predlist=torch.zeros(0,dtype=torch.long, device='cpu')
    lbllist=torch.zeros(0,dtype=torch.long, device='cpu')
    
    
    with torch.no_grad():
        for i, data in enumerate(dataloader):
            inputs, label = data[0].to(device), data[1].to(device)
            label_state = torch.zeros(len(label)).to(device)
            label_type = torch.zeros(len(label)).to(device)
            for k in range(len(label)):
                label_state[k] = relabel_state(label[k])
                label_type[k] = relabel_type(label[k])
                
            
            predictions_state, predictions_type = model(inputs)
            _, predicted_state = torch.max(predictions_state.data, 1)
            _,predicted_type = torch.max(predictions_type.data, 1)
    
            # Append batch prediction results
            predlist_state=torch.cat([predlist_state,predicted_state.view(-1).cpu()])
            lbllist_state=torch.cat([lbllist_state,label_state.view(-1).cpu()])
            predlist_type=torch.cat([predlist_type,predicted_type.view(-1).cpu()])
            lbllist_type=torch.cat([lbllist_type,label_type.view(-1).cpu()])
            
            predicted = torch.zeros(len(label)).to(device)
            # 15 label confusion matrix
            for k in range(len(predicted_state)):
                predicted[k] = recombine_prediction(predicted_state[k],predicted_type[k])
            predlist=torch.cat([predlist,predicted.view(-1).cpu()])
            lbllist=torch.cat([lbllist,label.view(-1).cpu()])
     
    # Confusion matrix and classification report
    conf_mat_state=confusion_matrix(lbllist_state.numpy(), predlist_state.numpy())
    conf_mat_type=confusion_matrix(lbllist_type.numpy(), predlist_type.numpy())
    conf_mat=confusion_matrix(lbllist.numpy(), predlist.numpy())
    class_report_state = classification_report(lbllist_state.numpy(), predlist_state.numpy())
    class_report_type = classification_report(lbllist_type.numpy(), predlist_type.numpy())
    class_report = classification_report(lbllist.numpy(), predlist.numpy())
    
    # print(conf_mat)
    # print(conf_mat_state)
    # print(conf_mat_type)
    print(class_report_state)
    print(class_report_type)
    print(class_report)
    
    # Per-class accuracy
    class_accuracy=100*conf_mat.diagonal()/conf_mat.sum(1)
    print(class_accuracy)
    class_accuracy_state=100*conf_mat_state.diagonal()/conf_mat_state.sum(1)
    print(class_accuracy_state)
    class_accuracy_type=100*conf_mat_type.diagonal()/conf_mat_type.sum(1)
    print(class_accuracy_type)
    return conf_mat_state, conf_mat_type, conf_mat, class_report_state, class_report_type, class_report

In [ ]:
model_loaded = torch.load(r"/content/drive/MyDrive/ECE324 project/Figures/Multi-Head/row 20/model.pt")
if confusion:
    conf_mat_state, conf_mat_type, conf_mat, class_report_state, class_report_type, class_report = confusion_classification_report(model_loaded, valid_loader)

    plt.figure()
    total_map = sns.heatmap(conf_mat,cmap="YlGnBu", annot=True,fmt='g',square = True,xticklabels=['ri_A', 'ri_B', 'ri_O', 'ri_R', 'ri_S', 'ro_A', 'ro_B', 'ro_O', 'ro_R', 'ro_S', 'ur_A', 'ur_B', 'ur_O', 'ur_R', 'ur_S'], yticklabels=['ri_A', 'ri_B', 'ri_O', 'ri_R', 'ri_S', 'ro_A', 'ro_B', 'ro_O', 'ro_R', 'ro_S', 'ur_A', 'ur_B', 'ur_O', 'ur_R', 'ur_S'] )
    bottom, top = total_map.get_ylim()
    total_map.set_ylim(bottom + 0.5, top - 0.5)
    total_map.set_title('Total Confusion Matrix')
    total_map.set_ylabel('True Label')
    total_map.set_xlabel('Predicted Label')
    plt.show()

    plt.figure()
    state_map = sns.heatmap(conf_mat_state,cmap="YlGnBu", annot=True,fmt='g',square = True,xticklabels=['Ripe', 'Rotten', 'Unripe'], yticklabels=['Ripe', 'Rotten', 'Unripe'] )
    bottom, top = state_map.get_ylim()
    state_map.set_ylim(bottom + 0.5, top - 0.5)
    state_map.set_title('State Confusion Matrix')
    state_map.set_ylabel('True Label')
    state_map.set_xlabel('Predicted Label')
    plt.show()

    plt.figure()
    type_map = sns.heatmap(conf_mat_type,cmap="YlGnBu", annot=True,fmt='g',square = True,xticklabels=['Apple','Banna','Orange','Raspberry','Strawberry'], yticklabels=['Apple','Banna','Orange','Raspberry','Strawberry'] )
    bottom, top = type_map.get_ylim()
    type_map.set_ylim(bottom + 0.5, top - 0.5)
    type_map.set_title('Type Confusion Matrix')
    type_map.set_ylabel('True Label')
    type_map.set_xlabel('Predicted Label')
    plt.show()

# **TEST SETS**

In [ ]:
test_white =  torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/ECE324 project/test_white_bg', transform=my_transform)
test_varied =  torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/ECE324 project/test_varied_bg', transform=my_transform)
test_green =  torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/ECE324 project/test_green_bg', transform=my_transform)

test_white_loader = DataLoader(test_white, batch_size=4)
test_varied_loader = DataLoader(test_varied, batch_size=4)

### **White background test set**

In [ ]:
running_loss = []
running_accuracy = []
running_loss_state = []
running_accuracy_state = []
running_loss_type = []
running_accuracy_type = []
for i, data in enumerate(test_white_loader):
        #get batch of data
    inputs, label = data[0].to(device), data[1].to(device)

    label_state = torch.zeros(len(label)).to(device)
    label_type = torch.zeros(len(label)).to(device)
    for k in range(len(label)):
        label_state[k] = relabel_state(label[k])
        label_type[k] = relabel_type(label[k]) 

    #run model on validation batch
    predictions_state, predictions_type = model_loaded(inputs)

        #compute loss
    batch_loss_state = loss_fnc(input=predictions_state.squeeze(), target=label_state.long())
    batch_loss_type = loss_fnc(input=predictions_type.squeeze(), target=label_type.long())

    Overall_loss = batch_loss_state + batch_loss_type
        #evaluate
    _, predicted_state = torch.max(predictions_state.data, 1)
    _,predicted_type = torch.max(predictions_type.data, 1)

    Acc_state = (label_state == predicted_state).sum().item() / 4
    Acc_type = (label_type == predicted_type).sum().item() / 4
    running_loss.append(Overall_loss.item())
    running_accuracy.append((Acc_state+Acc_type)/2)

    running_loss_state.append(batch_loss_state.item())
    running_accuracy_state.append(Acc_state)
    running_loss_type.append(batch_loss_type.item())
    running_accuracy_type.append(Acc_type) 
    
#Overall accuracy
whiteacc_ = sum(running_accuracy) / float(len(running_accuracy))
#state accuracy
whiteacc_state_ = sum(running_accuracy_state) / float(len(running_accuracy_state))

#type accuracy
whiteAcc_type_ = sum(running_accuracy_type) / float(len(running_accuracy_type))

#             lists for VALIDATION losses
#Overall Loss
white_loss_ = sum(running_loss) / float(len(running_loss))

#State Loss
white_loss_state_ = sum(running_loss_state) / float(len(running_loss_state))

#Type Loss
white_loss_type_ = sum(running_loss_type) / float(len(running_loss_type))

print(whiteacc_)
print(white_loss_)



### **Varied background test dataset**


In [ ]:
running_loss = []
running_accuracy = []
running_loss_state = []
running_accuracy_state = []
running_loss_type = []
running_accuracy_type = []
for i, data in enumerate(test_varied_loader):
        #get batch of data
    inputs, label = data[0].to(device), data[1].to(device)

    label_state = torch.zeros(len(label)).to(device)
    label_type = torch.zeros(len(label)).to(device)
    for k in range(len(label)):
        label_state[k] = relabel_state(label[k])
        label_type[k] = relabel_type(label[k]) 

    #run model on validation batch
    predictions_state, predictions_type = model_loaded(inputs)

        #compute loss
    batch_loss_state = loss_fnc(input=predictions_state.squeeze(), target=label_state.long())
    batch_loss_type = loss_fnc(input=predictions_type.squeeze(), target=label_type.long())

    Overall_loss = batch_loss_state + batch_loss_type
        #evaluate
    _, predicted_state = torch.max(predictions_state.data, 1)
    _,predicted_type = torch.max(predictions_type.data, 1)

    Acc_state = (label_state == predicted_state).sum().item() / 4
    Acc_type = (label_type == predicted_type).sum().item() / 4
    running_loss.append(Overall_loss.item())
    running_accuracy.append((Acc_state+Acc_type)/2)

    running_loss_state.append(batch_loss_state.item())
    running_accuracy_state.append(Acc_state)
    running_loss_type.append(batch_loss_type.item())
    running_accuracy_type.append(Acc_type) 
    
#Overall accuracy
variedacc_ = sum(running_accuracy) / float(len(running_accuracy))
#state accuracy
variedacc_state_ = sum(running_accuracy_state) / float(len(running_accuracy_state))

#type accuracy
variedAcc_type_ = sum(running_accuracy_type) / float(len(running_accuracy_type))

#             lists for VALIDATION losses
#Overall Loss
varied_loss_ = sum(running_loss) / float(len(running_loss))

#State Loss
varied_loss_state_ = sum(running_loss_state) / float(len(running_loss_state))

#Type Loss
varied_loss_type_ = sum(running_loss_type) / float(len(running_loss_type))

print(variedacc_)
print(varied_loss_)

### **General Test set**

In [ ]:
running_loss = []
running_accuracy = []
running_loss_state = []
running_accuracy_state = []
running_loss_type = []
running_accuracy_type = []
for i, data in enumerate(test_loader):
        #get batch of data
    inputs, label = data[0].to(device), data[1].to(device)

    label_state = torch.zeros(len(label)).to(device)
    label_type = torch.zeros(len(label)).to(device)
    for k in range(len(label)):
        label_state[k] = relabel_state(label[k])
        label_type[k] = relabel_type(label[k]) 

    #run model on validation batch
    predictions_state, predictions_type = model_loaded(inputs)

        #compute loss
    batch_loss_state = loss_fnc(input=predictions_state.squeeze(), target=label_state.long())
    batch_loss_type = loss_fnc(input=predictions_type.squeeze(), target=label_type.long())

    Overall_loss = batch_loss_state + batch_loss_type
        #evaluate
    _, predicted_state = torch.max(predictions_state.data, 1)
    _,predicted_type = torch.max(predictions_type.data, 1)

    Acc_state = (label_state == predicted_state).sum().item() / 64
    Acc_type = (label_type == predicted_type).sum().item() / 64
    running_loss.append(Overall_loss.item())
    running_accuracy.append((Acc_state+Acc_type)/2)

    running_loss_state.append(batch_loss_state.item())
    running_accuracy_state.append(Acc_state)
    running_loss_type.append(batch_loss_type.item())
    running_accuracy_type.append(Acc_type) 
    
#Overall accuracy
testacc_ = sum(running_accuracy) / float(len(running_accuracy))
#state accuracy
testacc_state_ = sum(running_accuracy_state) / float(len(running_accuracy_state))

#type accuracy
testAcc_type_ = sum(running_accuracy_type) / float(len(running_accuracy_type))

#             lists for VALIDATION losses
#Overall Loss
test_loss_ = sum(running_loss) / float(len(running_loss))

#State Loss
test_loss_state_ = sum(running_loss_state) / float(len(running_loss_state))

#Type Loss
test_loss_type_ = sum(running_loss_type) / float(len(running_loss_type))

print(testacc_)
print(test_loss_)